In [1]:
import pandas as pd
df_train = pd.read_csv('./weiboset/weibo_train_data.txt', delimiter = "\t", encoding="utf-8",header=None)
df_train.columns =['uid', 'mid', 'time', 'forward_count','comment_count','like_count','content']
df = pd.DataFrame(df_train)
df.columns =['uid', 'mid', 'time', 'forward_count','comment_count','like_count','content']
#df = df.rename(columns={"0": "uid", "1": "mid", "2": "time","3":"forward_count","4":"comment_count","5":"like_count","6":"content"})
df.head()

,uid,mid,time,forward_count,comment_count,like_count,content
0,d38e9bed5d98110dc2489d0d1cac3c2a,7d45833d9865727a88b960b0603c19f6,2015-02-23 17:41:29,0,0,0,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...
1,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,2015-02-14 12:49:58,0,0,0,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...
2,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,2015-03-31 13:58:06,0,0,0,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了
3,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,2015-06-11 20:39:57,0,4,3,看点不能说的，你们都懂[笑cry]
4,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,2015-03-10 18:02:38,0,0,0,111多张


In [2]:
context = df[['content']]
context.head()

,content
0,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...
1,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...
2,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了
3,看点不能说的，你们都懂[笑cry]
4,111多张


In [3]:
#Count mentions and Hastag.
context["count_mentions"] = context[['content']].astype(str).applymap(lambda x: str.count(x, '@'))
context["count_hastags"] = context[['content']].astype(str).applymap(lambda x: str.count(x, '#'))
context.head()

/home/jackcapt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jackcapt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,content,count_mentions,count_hastags
0,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,0,10
1,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...,1,2
2,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了,0,0
3,看点不能说的，你们都懂[笑cry],0,0
4,111多张,0,0


In [4]:
#Usable
import re
def clean_data(context):
#replace URL of a text
    #Remove links
    context['content'] = context['content'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '')
    #Remove Emoji
    context['content'] = context['content'].str.replace('\[.{0,12}\]', '')
    #Remove Hashtags
    tags = re.findall("#(.{0,30})#",str(context))
    context['content'] = context['content'].str.replace('#(.{0,30})#', '')
    #Extract&Remove EnglishCharacters
    english = re.findall("[a-z]+",str(context))
    context['content'] = context['content'].str.replace('[a-z]+', '')
    #Extract and remove Mentions
    at = re.findall("@([^@]{0,30})\s",str(context))
    context['content'] = context['content'].str.replace('@([^@]{0,30})\s', '')
    at+= re.findall("@([^@]{0,30})）",str(context))
    context['content'] = context['content'].str.replace('@([^@]{0,30})）', '')
    #Remove Spaces
    context['content'] = context['content'].str.replace('\s', '')
    #Remove Digits
    context['content'] = context['content'].str.replace('\d', '')
    #Remove……
    context['content'] = context['content'].str.replace('\.*', '')
    #Remove Punctuation
    context['content'] = context['content'].str.replace('[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）() ]+', '')
    context['content'] = context['content'].str.replace('[【】╮╯▽╰╭★→「」]+', '')
    context['content'] = context['content'].str.replace('！，❤。～《》：（）()【】「」？”“；：、', '')

clean_data(context)
print(context['content']);

/home/jackcapt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jackcapt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jackcapt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

0                                      丽江旅游推荐包赢股盈利对半分成不算本金群：
1          挣钱是一种能力抢红包拼的是技术我抢到了丁辰灵和一起发出的现金红包幸福感爆棚情人节一起来和粉丝...
2                                   淘宝网这些傻逼气的劳资有火没地儿发尼玛你们都瞎了
3                                                 看点不能说的你们都懂
4                                                         多张
5                                 有生之年我最喜欢的主跟我的三体勾搭到一起了幸福感爆棚
6                                           论优衣库试衣间隔音效果好坏分享自
7          如此平凡的日常一幕还能够再积累多少呢终有一天当我们到了看着这张照片能感受到一阵怀念的年纪时这...
8                              二十三糖瓜儿粘抢个红包乐翻天我抢到了罗永浩和你也来试试手气
9                          有好东西分享给你闪记笔记记事最好用的中文待办软件还等什么快去下载：
10         MAML用机器学习分析网络入侵日志的实例来练手“大数据“在安全领域也开始炒得火热了但大多数公...
11         生命会有许多风雨让我们痛但我们会在痛苦中成长而且喜欢喜欢能有这样的世界有乐也有痛；喜欢能有这...
12                                                准备测第一批集群数据
13         蛋糕奶油奶酪黄油牛奶拌匀;蛋黄打散牛奶面粉糖拌匀;和混匀;蛋白柠檬汁打发;预热电饭煲舀一半蛋...
14         武汉一专车司机因毒驾被治安拘留新华网武汉月日电记者冯国栋记者日从武汉市公安局获悉在日前一次执...
15                                       今年不管你做哪一行都看看这篇文章分享自
16                      

/home/jackcapt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
context.head()

,content,count_mentions,count_hastags
0,丽江旅游推荐包赢股盈利对半分成不算本金群：,0,10
1,挣钱是一种能力抢红包拼的是技术我抢到了丁辰灵和一起发出的现金红包幸福感爆棚情人节一起来和粉丝...,1,2
2,淘宝网这些傻逼气的劳资有火没地儿发尼玛你们都瞎了,0,0
3,看点不能说的你们都懂,0,0
4,多张,0,0


In [6]:
cleaned_data = pd.concat([df,context], axis=1, sort=False)
cleaned_data.head()

,uid,mid,time,forward_count,comment_count,like_count,content,content,count_mentions,count_hastags
0,d38e9bed5d98110dc2489d0d1cac3c2a,7d45833d9865727a88b960b0603c19f6,2015-02-23 17:41:29,0,0,0,丽江旅游(sz002033)#股票##炒股##财经##理财##投资#推荐包赢股，盈利对半分成...,丽江旅游推荐包赢股盈利对半分成不算本金群：,0,10
1,fa13974743d3fe6ff40d21b872325e9e,8169f1d45051e08ef213bf1106b1225d,2015-02-14 12:49:58,0,0,0,#丁辰灵的红包#挣钱是一种能力，抢红包拼的是技术。我抢到了丁辰灵 和@阚洪岩 一起发出的现金...,挣钱是一种能力抢红包拼的是技术我抢到了丁辰灵和一起发出的现金红包幸福感爆棚情人节一起来和粉丝...,1,2
2,da534fe87e7a52777bee5c30573ed5fd,68cd0258c31c2c525f94febea2d9523b,2015-03-31 13:58:06,0,0,0,淘宝网这些傻逼。。。气的劳资有火没地儿发~尼玛，你们都瞎了,淘宝网这些傻逼气的劳资有火没地儿发尼玛你们都瞎了,0,0
3,e06a22b7e065e559a1f0bf7841a85c51,00b9f86b4915aedb7db943c54fd19d59,2015-06-11 20:39:57,0,4,3,看点不能说的，你们都懂[笑cry],看点不能说的你们都懂,0,0
4,f9828598f9664d4e347ef2048ce17734,c7f6f66044c0c5a3330e2c5371be6824,2015-03-10 18:02:38,0,0,0,111多张,多张,0,0


In [7]:
#Export CSV
#cleaned_data.to_csv('./df_tranClean.csv', index=False)

In [8]:
#Count symbol functions.
#from pandas import *
#import re

#df_H = DataFrame({"emails": ["fake@gmail.com, example@gmail.com","KingArthur@aol.com, none, SirRobyn@msn.net, TheBlackKnight@clintonserver.com"]})
#context

#at = re.compile(r"@", re.I)
#def count_mention(string):
#    count = 0
#    for i in at.finditer(str(string)):
#        count += 1
#    return count

#context["count_mention"] = context["content"].map(count_mention)

#at = re.compile(r"#(.{0,30})#", re.I)
#def count_mention(string):
#    count = 0
#    for i in at.finditer(str(string)):
#        count += 1
#    return count

#context["count_Hashtag"] = context["content"].map(count_mention)
#context